# Project of ana and leo

In [1]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from function import Utils
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas.api.types import is_numeric_dtype
import numpy as np
from IPython.display import display

warnings.filterwarnings('ignore')

## Data loading

In [84]:
df = pd.read_csv("hmw/small3insert.csv")

In [85]:
df.head()

,Case,Event,Activity,Timestamp,Resource,Resource_failure_rate,Resource_Pass/Fail,order,resource_anomaly_type,resource_parameter,trace_change_resource,variant_num
0,case_0,9367,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,0.026984,0,1,normal,NaN,0,var_358
1,case_0,9368,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,0.018454,0,2,normal,NaN,0,var_358
2,case_0,9369,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,0.002551,0,3,normal,NaN,0,var_358
3,case_0,9370,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,0.029775,0,4,normal,NaN,0,var_358
4,case_0,9371,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_358


## Data preprocessing

In [88]:
input = {
    "Case" : "caseid",
    "Activity" : "activity" ,
    "Resource" : "resource" ,
    "Timestamp": "ts",
    "resource_anomaly_type" : "y"  #Needs to be 1 for each Case
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

df["dt"] = Utils.dt(df) # Thisneed to be improved
df.drop("ts",axis=1,inplace = True)

In [89]:
Utils.persantage_nan(df)

caseid                    0.000000
Event                     0.000000
activity                  0.000000
resource                  0.000000
Resource_failure_rate     0.000000
Resource_Pass/Fail        0.000000
order                     0.000000
y                         0.000000
resource_parameter       83.957511
trace_change_resource     0.000000
variant_num               0.000000
dt                        0.000000
dtype: float64

In [90]:
df = Utils.predict_null_value(colum="resource_parameter",df=df)

In [91]:
df = Utils.prod_nan_with_treshold(df)
df.head()

,caseid,Event,activity,resource,Resource_failure_rate,Resource_Pass/Fail,order,y,resource_parameter,trace_change_resource,variant_num,dt,resource_parameter_was_null
0,case_0,9367,Activity A,Resource_Group2_res_1,0.026984,0,1,normal,"loc = 5, len = 1.0",0,var_358,0 days 01:00:00,True
1,case_0,9368,Activity B,Resource_Group0_res_0,0.018454,0,2,normal,"loc = 7, len = 3.0",0,var_358,0 days 01:00:00,True
2,case_0,9369,Activity C,Resource_Group1_res_0,0.002551,0,3,normal,"loc = 7, len = 3.0",0,var_358,0 days 01:00:00,True
3,case_0,9370,Activity D,Resource_Group2_res_2,0.029775,0,4,normal,"loc = 5, len = 1.0",0,var_358,0 days 01:00:00,True
4,case_0,9371,Activity E,Resource_Group0_res_0,0.018454,0,5,normal,"loc = 7, len = 3.0",0,var_358,0 days 01:00:00,True


In [92]:
df_grouped = df.groupby(["caseid"]).agg(list).reset_index()

In [93]:
df_grouped = Utils.reduce_list_columns(df_grouped)

In [94]:
Utils.display_columns_to_aggragate(df_grouped)

,Event,activity,resource,Resource_failure_rate,Resource_Pass/Fail,order,resource_parameter,dt
0,"[9367, 9368, 9369, 9370, 9371, 9372, 9373]","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...","[0.0269841320197102, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7]","[loc = 5, len = 1.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
1,"[13225, 13226, 13227, 13227, 13227, 13228, 132...","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.007...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[loc = 3, len = 2.0, loc = 3, len = 2.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 00:0..."
2,"[1025, 1026, 1027, 1028, 1029, 1030, 1031]","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...","[0.0269841320197102, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7]","[loc = 5, len = 1.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
3,"[39553, 39554, 39555, 39556, 39557, 39558, 395...","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...","[0.0269841320197102, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 5, len = 2.0, loc = 5, len = 2.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4,"[27951, 27952, 27953, 27954, 27955, 27956, 279...","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 5, len = 2.0, loc = 5, len = 1.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
...,...,...,...,...,...,...,...,...
4995,"[11080, 11081, 11082, 11083, 11084, 11085, 110...","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 7, len = 3.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4996,"[9536, 9537, 9538, 9539, 9540, 9541, 9542, 954...","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_2, Resource_Group0_res_0,...","[0.029775339741158, 0.0184536333969282, 0.0070...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 2, len = 2.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4997,"[7567, 7568, 7569, 7570, 7571, 7572, 7573, 757...","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.007...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 7, len = 3.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4998,"[8949, 8950, 8951, 8952, 8953, 8954, 8955, 895...","[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.007...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 7, len = 3.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."


In [98]:
#Some colomns have no importance in the process so we drop them
df_grouped = df_grouped.drop(['Event','order'],axis=1)
df_grouped = df_grouped.drop(['Resource_failure_rate','Resource_Pass/Fail'],axis=1)

In [99]:
df_grouped.head(2)

,caseid,activity,resource,y,resource_parameter,trace_change_resource,variant_num,dt,resource_parameter_was_null
0,case_0,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...",normal,"[loc = 5, len = 1.0, loc = 7, len = 3.0, loc =...",0,var_358,"[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0...",True
1,case_1,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...",insert,"[loc = 3, len = 2.0, loc = 3, len = 2.0, loc =...",1,var_406,"[0 days 01:00:00, 0 days 01:00:00, 0 days 00:0...",False


In [97]:
df_grouped_agg = Utils.aggregation_encoding(df_grouped,df)

In [100]:
df_grouped_t = df_grouped.copy()
    
for col in df_grouped_t.columns:    
    #the columns needs to have only list elements
    x = df_grouped_t[col].iloc[0]
    if(type(x) == list and col != "dt"):
        print(col)
        for el in df[col].unique():
            df_grouped_t[el] = df_grouped_t[col].map(lambda x : x.count(el)/len(x))
        df_grouped_t.drop(col,axis=1,inplace = True)

        
df_grouped_t["avg_dt"] = df_grouped_t["dt"].map(lambda x : np.average(x))
df_grouped_t["max_dt"] = df_grouped_t["dt"].map(lambda x : np.max(x))
df_grouped_t["min_dt"] = df_grouped_t["dt"].map(lambda x : np.min(x))
df_grouped_t.drop("dt",axis=1,inplace = True)

a =  df_grouped_t.copy()

activity
resource
resource_parameter


## Logs encoding

i think: activity - onehot?, variant - one hot, action, onehot, event origin - onehot, liofecycle - one hot

## Model selection

## Evaluation of results